# Imports

In [8]:
# Imports

import requests # HTTP requests
from bs4 import BeautifulSoup # Extract HTML content

import pandas as pd
import numpy as np

# Title scraper

In [9]:
def scrape_tech_news():

    url = 'https://www.ft.com/technology'

    # HTTP request to scrap page information
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)

    # Check response status
    if response.status_code != 200:
        print(f"Échec du scraping : {response.status_code}")
        return

    # HTML content analysis
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all the article listed on the page
    articles = soup.find_all('div', class_='o-teaser__content')

    news = []

    for article in articles:
        # Title and link extraction
        article_heading = article.find('a', class_='js-teaser-heading-link')
        title = article_heading.get_text(strip=True) if article_heading else "No title"
        link = "https://www.ft.com" + article_heading['href'] if article_heading else "No link"

        # Extract article tag to define categories
        article_tag = article.find('a', class_='o-teaser__tag')
        tag = article_tag['aria-label'] if article_tag else 'No cat'
        tag = tag.replace('Category: ', '')

        news.append({
            'title': title,
            'link': link,
            'tag': tag,
        })

    return news

ft_news_scraped = scrape_tech_news()

# Print
for article in ft_news_scraped[0:2]:
    print(f"Title: {article['title']}")
    print(f"Link: {article['link']}")
    print(f"Category: {article['tag']}")
    print("-" * 40)


Title: Founder Anne Wojcicki races to rescue 23andMe
Link: https://www.ft.com/content/7751bc99-132c-407d-ac97-626fbb01855b
Category: Biotech
----------------------------------------
Title: ASML shares drop sharply after warning on semiconductor recovery
Link: https://www.ft.com/content/3b11018f-960d-442f-8dc7-ed6b555d87f6
Category: ASML Holding NV
----------------------------------------


In [10]:
# Save the titles we want to generate the content for

In [11]:
ft_news_scraped = pd.DataFrame(ft_news_scraped)

In [12]:
ft_news_scraped.to_csv('../articles/titles.csv')

Parfait